<a href="https://colab.research.google.com/github/saranoor/Practice/blob/main/bert_shap_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet shap==0.39
!pip install --quiet transformers==4.5.0

import shap
import transformers

from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          TextClassificationPipeline)

     |████████████████████████████████| 356 kB 5.3 MB/s 
     |████████████████████████████████| 2.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 35.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
#model_name = "urduhack/roberta-urdu-small"
tokenizer = AutoTokenizer.from_pretrained('urduhack/roberta-urdu-small')
model=torch.load('drive/My Drive/Thesis/my_depression_model_t',map_location=lambda storage, loc: storage)
# model = AutoModelForSequenceClassification.from_pretrained(
#     "drive/My Drive/Thesis/my_depression_model_t"
#     )
#pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

In [4]:
next(model.parameters()).is_cuda

False

In [5]:
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [6]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)

In [7]:
def score_and_visualize(text):
  prediction = pipe([text])
  print(prediction[0])

  explainer = shap.Explainer(pipe)
  shap_values = explainer([text])

  shap.plots.text(shap_values)

In [9]:
#score_and_visualize("میں ہوں کہ میں کامیاب ہوں گی")

In [8]:
params = list(model.named_parameters())

In [9]:
print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (52000, 768)
roberta.embeddings.position_embeddings.weight             (514, 768)
roberta.embeddings.token_type_embeddings.weight             (1, 768)
roberta.embeddings.LayerNorm.weight                           (768,)
roberta.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight       (768, 768)
roberta.encoder.layer.0.attention.self.query.bias             (768,)
roberta.encoder.layer.0.attention.self.key.weight         (768, 768)
roberta.encoder.layer.0.attention.self.key.bias               (768,)
roberta.encoder.layer.0.attention.self.value.weight       (768, 768)
roberta.encoder.layer.0.attention.self.value.bias             (768,)
roberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
roberta.encoder.layer.0.attention.output.dense.bias           (

In [10]:
text="میں ہوں کہ میں کامیاب ہوں گی"
prediction = pipe([text])
print(prediction[0])

explainer = shap.Explainer(pipe)
shap_values = explainer([text])

#shap.plots.text(shap_values)
shap.plots.text(shap_values[0,:,"LABEL_1"])

[{'label': 'LABEL_0', 'score': 0.02810482867062092}, {'label': 'LABEL_1', 'score': 0.971895158290863}]


In [11]:
shap_values

.values =
array([[[ 2.42776106e-07, -5.22896556e-07],
        [-2.44770849e-01,  2.44770885e-01],
        [-2.35465048e-01,  2.35464974e-01],
        [ 1.85384567e-02, -1.85389758e-02],
        [-2.27078389e-01,  2.27078926e-01],
        [-6.96204567e-01,  6.96204204e-01],
        [-1.61140152e-01,  1.61140401e-01],
        [-1.14130784e-01,  1.14130580e-01],
        [-1.23441088e-07,  5.22896556e-07]]])

.base_values =
array([[-1.88305534,  1.88305508]])

.data =
array([['', 'میں ', 'ہوں ', 'کہ ', 'میں ', 'کامیاب ', 'ہوں ', 'گی', '']],
      dtype='<U7')